<a href="https://colab.research.google.com/github/dominikacecylia/Automatic-post-editing/blob/main/MULTI_Shared_Attention_for_APE.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# memory footprint support libraries/code
!ln -sf /opt/bin/nvidia-smi /usr/bin/nvidia-smi
!pip install gputil
!pip install psutil
!pip install humanize
import psutil
import humanize
import os
import GPUtil as GPU
GPUs = GPU.getGPUs()
# XXX: only one GPU on Colab and isn’t guaranteed
gpu = GPUs[0]
def printm():
    process = psutil.Process(os.getpid())
    print("CPU RAM Free: " + humanize.naturalsize( psutil.virtual_memory().available ), " | Proc size: " + humanize.naturalsize( process.memory_info().rss))
    print("GPU RAM Free: {0:.0f}MB | Used: {1:.0f}MB | Util {2:3.0f}% | Total {3:.0f}MB".format(gpu.memoryFree, gpu.memoryUsed, gpu.memoryUtil*100, gpu.memoryTotal))
printm()

  Created wheel for gputil: filename=GPUtil-1.4.0-cp37-none-any.whl size=7411 sha256=0c34da74382684b52f2d978702a84803a26903ddfffe9360f4797566b5123765
  Stored in directory: /root/.cache/pip/wheels/3d/77/07/80562de4bb0786e5ea186911a2c831fdd0018bda69beab71fd
Successfully built gputil
CPU RAM Free: 12.8 GB  | Proc size: 111.8 MB
GPU RAM Free: 16280MB | Used: 0MB | Util   0% | Total 16280MB


In [ ]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Select the Runtime > "Change runtime type" menu to enable a GPU accelerator, ')
  print('and then re-execute this cell.')
else:
  print(gpu_info)

Tue Mar  2 20:39:32 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   35C    P0    27W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
! pip uninstall fastai -y
! pip uninstall en-core-web-sm -y
! pip uninstall fbprophet -y
! pip uninstall multiprocess -y

In [ ]:
!python3 --version

Python 3.7.10


In [ ]:
!pip uninstall tensorflow -y

Uninstalling tensorflow-1.15.2:
  Successfully uninstalled tensorflow-1.15.2


In [ ]:
! pip install tensorflow==1.15.0
! pip install spacy==2.0.5
! pip install six
! pip install tqdm==4.19.9
! pip install torchtext==0.1.1
! pip install future
! pip install torchvision==0.2.2
! pip install torch==0.1.2

Processing /root/.cache/pip/wheels/b1/c2/ed/d62208260edbd3fa7156545c00ef966f45f2063d0a84f8208a/wrapt-1.12.1-cp37-cp37m-linux_x86_64.whl
  Found existing installation: wrapt 1.10.11
    Uninstalling wrapt-1.10.11:
      Successfully uninstalled wrapt-1.10.11
Processing /root/.cache/pip/wheels/e0/9e/5c/fcc58667983ae85a26aa032887eaab03b537bf8c3e0bf49cfe/spacy-2.0.5-cp37-cp37m-linux_x86_64.whl
  Using cached https://files.pythonhosted.org/packages/94/b1/47a88072d0a38b3594c0a638a62f9ef7c742b8b8a87f7b105f7ed720b14b/thinc-6.10.3.tar.gz
Processing /root/.cache/pip/wheels/48/5d/04/22361a593e70d23b1f7746d932802efe1f0e523376a74f321e/wrapt-1.10.11-cp37-cp37m-linux_x86_64.whl
  ERROR: Failed building wheel for thinc
  Running setup.py clean for thinc
Failed to build thinc
ERROR: tensorflow 1.15.0 has requirement wrapt>=1.11.1, but you'll have wrapt 1.10.11 which is incompatible.
  Found existing installation: wrapt 1.12.1
    Uninstalling wrapt-1.12.1:
      Successfully uninstalled wrapt-1.12.1
  

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


Source of algorithm: https://github.com/modernmt/modernmt

### **Pre-processing**

In [ ]:
! python  ./drive/MyDrive/Shared_Attention_for_APE-master/preprocess.py  -train_src  ./drive/MyDrive/sp_train.src  -train_inter  ./drive/MyDrive/pl_train.mt  -train_tgt  ./drive/MyDrive/pl_train.pe  -valid_src  ./drive/MyDrive/test/sp_dev.src  -valid_inter  ./drive/MyDrive/test/pl_dev.mt  -valid_tgt  ./drive/MyDrive/test/pl_dev.mt  -save_data  ./drive/MyDrive/Shared_Attention_for_APE-master/output  -lower

The vocabs are learned
Preparing training ...
Building Training...
Building Vocab...
Building Valid...
Saving train/valid/fields


# **Training**

## **Default setup**

In [ ]:
! python  ./drive/MyDrive/Shared_Attention_for_APE-master/train.py  -encoder_type  double_encoder  -data  ./drive/MyDrive/Shared_Attention_for_APE-master/output  -save_model  ./drive/MyDrive/test  -word_vec_size  200  -epochs  13 -gpuid 0 -batch_size 64

Loading data from './drive/MyDrive/Shared_Attention_for_APE-master/output'
 * vocabulary size. source = 50002; inter = 50002; target = 50004
 * number of training sentences. 77383
 * maximum batch size. 64
Building model...
The double encoder will be needed here
Bidirectional RNN!
DoubleEncNMTModel(
  (encoder): DoubleRNNEncoder(
    (embeddings_src): Embeddings(
      (make_embedding): Sequential(
        (emb_luts): Elementwise(
          (0): Embedding(50002, 200, padding_idx=1)
        )
      )
    )
    (embeddings_inter): Embeddings(
      (make_embedding): Sequential(
        (emb_luts): Elementwise(
          (0): Embedding(50002, 200, padding_idx=1)
        )
      )
    )
    (rnn1): LSTM(200, 250, num_layers=2, dropout=0.3, bidirectional=True)
    (rnn2): LSTM(200, 250, num_layers=2, dropout=0.3, bidirectional=True)
  )
  (decoder): InputFeedRNNDecoder(
    (embeddings): Embeddings(
      (make_embedding): Sequential(
        (emb_luts): Elementwise(
          (0): Embeddin

In [ ]:
! python  ./drive/MyDrive/Shared_Attention_for_APE-master/translate.py  -model  ./drive/MyDrive/test_acc_47.06_ppl_46.11_e13.pt  -src  ./drive/MyDrive/test/sp_test.src  -inter  ./drive/MyDrive/test/pl_test.mt -tgt ./drive/MyDrive/test/pl_test.pe  -output  ./drive/MyDrive/Shared_Attention_for_APE-master/prediction_default  -replace_unk

The double encoder will be needed here
Bidirectional RNN!
Loading model
make_src has been called!!!
/content/drive/MyDrive/Shared_Attention_for_APE-master/onmt/modules/GlobalAttention.py:150: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  align_vectors = self.sm(align.view(batch*targetL, sourceL))
/usr/local/lib/python3.6/dist-packages/torch/nn/modules/container.py:67: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  input = module(input)
/usr/local/lib/python3.6/dist-packages/torch/tensor.py:309: UserWarning: other is not broadcastable to self, but they have the same number of elements.  Falling back to deprecated pointwise behavior.
  return self.add_(other)
make_src has been called!!!
make_src has been called!!!
make_src has been called!!!
make_src has been called!!!
make_src has been called!!!
make_src has been called!!!
make_src 

In [ ]:
! python  ./drive/MyDrive/Shared_Attention_for_APE-master/translate.py  -model  ./drive/MyDrive/MultiSharedAttmodels/test_acc_47.06_ppl_46.11_e13.pt  -src  ./multi_src_60.txt  -inter  ./multi_mt_60.txt -tgt ./multi_pe_60.txt  -output  ./drive/MyDrive/Shared_Attention_for_APE-master/prediction_default_short  -replace_unk

The double encoder will be needed here
Bidirectional RNN!
Loading model
/usr/local/lib/python3.7/dist-packages/torchtext/data/field.py:197: UserWarning: volatile was removed and now has no effect. Use `with torch.no_grad():` instead.
  return Variable(arr, volatile=not train), lengths
/usr/local/lib/python3.7/dist-packages/torchtext/data/field.py:198: UserWarning: volatile was removed and now has no effect. Use `with torch.no_grad():` instead.
  return Variable(arr, volatile=not train)
make_src has been called!!!
/content/drive/MyDrive/Shared_Attention_for_APE-master/onmt/Translator.py:135: UserWarning: volatile was removed and now has no effect. Use `with torch.no_grad():` instead.
  def var(a): return Variable(a, volatile=True)
/content/drive/MyDrive/Shared_Attention_for_APE-master/onmt/Models.py:653: UserWarning: volatile was removed and now has no effect. Use `with torch.no_grad():` instead.
  for e in self._all]
/content/drive/MyDrive/Shared_Attention_for_APE-master/onmt/modules/G

## **Experiment with batch size** 

In [ ]:
! python  ./drive/MyDrive/Shared_Attention_for_APE-master/train.py  -encoder_type  double_encoder  -data  ./drive/MyDrive/Shared_Attention_for_APE-master/output  -save_model  ./drive/MyDrive/test  -word_vec_size  200  -epochs  13 -gpuid 0 -batch_size 32

Loading data from './drive/MyDrive/Shared_Attention_for_APE-master/output'
 * vocabulary size. source = 50002; inter = 50002; target = 50004
 * number of training sentences. 77383
 * maximum batch size. 32
Building model...
The double encoder will be needed here
Bidirectional RNN!
DoubleEncNMTModel(
  (encoder): DoubleRNNEncoder(
    (embeddings_src): Embeddings(
      (make_embedding): Sequential(
        (emb_luts): Elementwise(
          (0): Embedding(50002, 200, padding_idx=1)
        )
      )
    )
    (embeddings_inter): Embeddings(
      (make_embedding): Sequential(
        (emb_luts): Elementwise(
          (0): Embedding(50002, 200, padding_idx=1)
        )
      )
    )
    (rnn1): LSTM(200, 250, num_layers=2, dropout=0.3, bidirectional=True)
    (rnn2): LSTM(200, 250, num_layers=2, dropout=0.3, bidirectional=True)
  )
  (decoder): InputFeedRNNDecoder(
    (embeddings): Embeddings(
      (make_embedding): Sequential(
        (emb_luts): Elementwise(
          (0): Embeddin

In [ ]:
! python  ./drive/MyDrive/Shared_Attention_for_APE-master/translate.py  -model  ./drive/MyDrive/test_acc_51.04_ppl_36.35_e12.pt  -src  ./drive/MyDrive/test/sp_test.src  -inter  ./drive/MyDrive/test/pl_test.mt -tgt ./drive/MyDrive/test/pl_test.pe  -output  ./drive/MyDrive/Shared_Attention_for_APE-master/prediction_file_batch32  -replace_unk

The double encoder will be needed here
Bidirectional RNN!
Loading model
make_src has been called!!!
/content/drive/MyDrive/Shared_Attention_for_APE-master/onmt/modules/GlobalAttention.py:150: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  align_vectors = self.sm(align.view(batch*targetL, sourceL))
/usr/local/lib/python3.6/dist-packages/torch/nn/modules/container.py:67: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  input = module(input)
/usr/local/lib/python3.6/dist-packages/torch/tensor.py:309: UserWarning: other is not broadcastable to self, but they have the same number of elements.  Falling back to deprecated pointwise behavior.
  return self.add_(other)
make_src has been called!!!
make_src has been called!!!
make_src has been called!!!
make_src has been called!!!
make_src has been called!!!
make_src has been called!!!
make_src 

## **Experiment with bach size and epochs**

In [ ]:
! python  ./drive/MyDrive/Shared_Attention_for_APE-master/train.py  -encoder_type  double_encoder  -data  ./drive/MyDrive/Shared_Attention_for_APE-master/output  -save_model  ./drive/MyDrive/test  -word_vec_size  200  -epochs  20 -gpuid 0 -batch_size 32 

Loading data from './drive/MyDrive/Shared_Attention_for_APE-master/output'
 * vocabulary size. source = 50002; inter = 50002; target = 50004
 * number of training sentences. 77383
 * maximum batch size. 32
Building model...
The double encoder will be needed here
Bidirectional RNN!
DoubleEncNMTModel(
  (encoder): DoubleRNNEncoder(
    (embeddings_src): Embeddings(
      (make_embedding): Sequential(
        (emb_luts): Elementwise(
          (0): Embedding(50002, 200, padding_idx=1)
        )
      )
    )
    (embeddings_inter): Embeddings(
      (make_embedding): Sequential(
        (emb_luts): Elementwise(
          (0): Embedding(50002, 200, padding_idx=1)
        )
      )
    )
    (rnn1): LSTM(200, 250, num_layers=2, dropout=0.3, bidirectional=True)
    (rnn2): LSTM(200, 250, num_layers=2, dropout=0.3, bidirectional=True)
  )
  (decoder): InputFeedRNNDecoder(
    (embeddings): Embeddings(
      (make_embedding): Sequential(
        (emb_luts): Elementwise(
          (0): Embeddin

In [ ]:
! python  ./drive/MyDrive/Shared_Attention_for_APE-master/translate.py  -model  ./drive/MyDrive/test_acc_48.90_ppl_41.26_e20.pt  -src  ./drive/MyDrive/test/sp_test.src  -inter  ./drive/MyDrive/test/pl_test.mt -tgt ./drive/MyDrive/test/pl_test.pe  -output  ./drive/MyDrive/Shared_Attention_for_APE-master/prediction_file_batch32epoch  -replace_unk

The double encoder will be needed here
Bidirectional RNN!
Loading model
make_src has been called!!!
/content/drive/MyDrive/Shared_Attention_for_APE-master/onmt/modules/GlobalAttention.py:150: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  align_vectors = self.sm(align.view(batch*targetL, sourceL))
/usr/local/lib/python3.6/dist-packages/torch/nn/modules/container.py:67: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  input = module(input)
/usr/local/lib/python3.6/dist-packages/torch/tensor.py:309: UserWarning: other is not broadcastable to self, but they have the same number of elements.  Falling back to deprecated pointwise behavior.
  return self.add_(other)
make_src has been called!!!
make_src has been called!!!
make_src has been called!!!
make_src has been called!!!
make_src has been called!!!
make_src has been called!!!
make_src 

## **Experiments with optim**

### Adam

In [ ]:
! python  ./drive/MyDrive/Shared_Attention_for_APE-master/train.py  -encoder_type  double_encoder  -data  ./drive/MyDrive/Shared_Attention_for_APE-master/output  -save_model  ./drive/MyDrive/test  -word_vec_size  200  -epochs  13 -gpuid 0 -batch_size 64 -optim adam 

Loading data from './drive/MyDrive/Shared_Attention_for_APE-master/output'
 * vocabulary size. source = 50002; inter = 50002; target = 50004
 * number of training sentences. 77383
 * maximum batch size. 64
Building model...
The double encoder will be needed here
Bidirectional RNN!
DoubleEncNMTModel(
  (encoder): DoubleRNNEncoder(
    (embeddings_src): Embeddings(
      (make_embedding): Sequential(
        (emb_luts): Elementwise(
          (0): Embedding(50002, 200, padding_idx=1)
        )
      )
    )
    (embeddings_inter): Embeddings(
      (make_embedding): Sequential(
        (emb_luts): Elementwise(
          (0): Embedding(50002, 200, padding_idx=1)
        )
      )
    )
    (rnn1): LSTM(200, 250, num_layers=2, dropout=0.3, bidirectional=True)
    (rnn2): LSTM(200, 250, num_layers=2, dropout=0.3, bidirectional=True)
  )
  (decoder): InputFeedRNNDecoder(
    (embeddings): Embeddings(
      (make_embedding): Sequential(
        (emb_luts): Elementwise(
          (0): Embeddin

In [ ]:
! python  ./drive/MyDrive/Shared_Attention_for_APE-master/translate.py  -model  ./drive/MyDrive/test_acc_19.95_ppl_26881171418161356094253400435962903554686976.00_e13.pt  -src  ./drive/MyDrive/test/sp_test.src  -inter  ./drive/MyDrive/test/pl_test.mt -tgt ./drive/MyDrive/test/pl_test.pe  -output  ./drive/MyDrive/Shared_Attention_for_APE-master/prediction_file_adam  -replace_unk

The double encoder will be needed here
Bidirectional RNN!
Loading model
make_src has been called!!!
/content/drive/MyDrive/Shared_Attention_for_APE-master/onmt/modules/GlobalAttention.py:150: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  align_vectors = self.sm(align.view(batch*targetL, sourceL))
/usr/local/lib/python3.6/dist-packages/torch/nn/modules/container.py:67: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  input = module(input)
/usr/local/lib/python3.6/dist-packages/torch/tensor.py:309: UserWarning: other is not broadcastable to self, but they have the same number of elements.  Falling back to deprecated pointwise behavior.
  return self.add_(other)
make_src has been called!!!
make_src has been called!!!
make_src has been called!!!
make_src has been called!!!
make_src has been called!!!
make_src has been called!!!
make_src 

### Adagrad

In [ ]:
! python  ./drive/MyDrive/Shared_Attention_for_APE-master/train.py  -encoder_type  double_encoder  -data  ./drive/MyDrive/Shared_Attention_for_APE-master/output  -save_model  ./drive/MyDrive/test  -word_vec_size  300  -epochs  20  -batch_size 32 -optim adagrad #-gpuid 0

^C


In [ ]:
! python  ./drive/MyDrive/Shared_Attention_for_APE-master/translate.py  -model  ./drive/MyDrive/test_acc_19.95_ppl_1345663766941117183950848.00_e20.pt  -src  ./drive/MyDrive/test/sp_test.src  -inter  ./drive/MyDrive/test/pl_test.mt -tgt ./drive/MyDrive/test/pl_test.pe  -output  ./drive/MyDrive/Shared_Attention_for_APE-master/prediction_file_adagrad  -replace_unk

### Adadelta

In [ ]:
! python  ./drive/MyDrive/Shared_Attention_for_APE-master/train.py  -encoder_type  double_encoder  -data  ./drive/MyDrive/Shared_Attention_for_APE-master/output  -save_model  ./drive/MyDrive/test  -word_vec_size  200  -epochs  20 -gpuid 0 -batch_size 32 -optim adadelta

Loading data from './drive/MyDrive/Shared_Attention_for_APE-master/output'
 * vocabulary size. source = 50002; inter = 50002; target = 50004
 * number of training sentences. 77383
 * maximum batch size. 32
Building model...
The double encoder will be needed here
Bidirectional RNN!
DoubleEncNMTModel(
  (encoder): DoubleRNNEncoder(
    (embeddings_src): Embeddings(
      (make_embedding): Sequential(
        (emb_luts): Elementwise(
          (0): Embedding(50002, 200, padding_idx=1)
        )
      )
    )
    (embeddings_inter): Embeddings(
      (make_embedding): Sequential(
        (emb_luts): Elementwise(
          (0): Embedding(50002, 200, padding_idx=1)
        )
      )
    )
    (rnn1): LSTM(200, 250, num_layers=2, dropout=0.3, bidirectional=True)
    (rnn2): LSTM(200, 250, num_layers=2, dropout=0.3, bidirectional=True)
  )
  (decoder): InputFeedRNNDecoder(
    (embeddings): Embeddings(
      (make_embedding): Sequential(
        (emb_luts): Elementwise(
          (0): Embeddin

In [ ]:
! python  ./drive/MyDrive/Shared_Attention_for_APE-master/translate.py  -model  ./drive/MyDrive/test_acc_64.49_ppl_14.74_e19.pt  -src  ./drive/MyDrive/test/sp_test.src  -inter  ./drive/MyDrive/test/pl_test.mt -tgt ./drive/MyDrive/test/pl_test.pe  -output  ./drive/MyDrive/Shared_Attention_for_APE-master/prediction_adadelta  -replace_unk

The double encoder will be needed here
Bidirectional RNN!
Loading model
make_src has been called!!!
/content/drive/MyDrive/Shared_Attention_for_APE-master/onmt/modules/GlobalAttention.py:150: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  align_vectors = self.sm(align.view(batch*targetL, sourceL))
/usr/local/lib/python3.6/dist-packages/torch/nn/modules/container.py:67: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  input = module(input)
/usr/local/lib/python3.6/dist-packages/torch/tensor.py:309: UserWarning: other is not broadcastable to self, but they have the same number of elements.  Falling back to deprecated pointwise behavior.
  return self.add_(other)
make_src has been called!!!
make_src has been called!!!
make_src has been called!!!
make_src has been called!!!
make_src has been called!!!
make_src has been called!!!
make_src 

#### bigger word vector

In [ ]:
! python  ./drive/MyDrive/Shared_Attention_for_APE-master/train.py  -encoder_type  double_encoder  -data  ./drive/MyDrive/Shared_Attention_for_APE-master/output  -save_model  ./drive/MyDrive/test  -word_vec_size  300  -epochs  20 -gpuid 0 -batch_size 32 -optim adadelta

Loading data from './drive/MyDrive/Shared_Attention_for_APE-master/output'
 * vocabulary size. source = 50002; inter = 50002; target = 50004
 * number of training sentences. 77383
 * maximum batch size. 32
Building model...
The double encoder will be needed here
Bidirectional RNN!
DoubleEncNMTModel(
  (encoder): DoubleRNNEncoder(
    (embeddings_src): Embeddings(
      (make_embedding): Sequential(
        (emb_luts): Elementwise(
          (0): Embedding(50002, 300, padding_idx=1)
        )
      )
    )
    (embeddings_inter): Embeddings(
      (make_embedding): Sequential(
        (emb_luts): Elementwise(
          (0): Embedding(50002, 300, padding_idx=1)
        )
      )
    )
    (rnn1): LSTM(300, 250, num_layers=2, dropout=0.3, bidirectional=True)
    (rnn2): LSTM(300, 250, num_layers=2, dropout=0.3, bidirectional=True)
  )
  (decoder): InputFeedRNNDecoder(
    (embeddings): Embeddings(
      (make_embedding): Sequential(
        (emb_luts): Elementwise(
          (0): Embeddin

In [ ]:
! python  ./drive/MyDrive/Shared_Attention_for_APE-master/translate.py  -model  ./drive/MyDrive/test_acc_64.58_ppl_13.82_e20.pt  -src  ./drive/MyDrive/test/sp_test.src  -inter  ./drive/MyDrive/test/pl_test.mt -tgt ./drive/MyDrive/test/pl_test.pe  -output  ./drive/MyDrive/Shared_Attention_for_APE-master/prediction_adadelta2  -replace_unk

The double encoder will be needed here
Bidirectional RNN!
Loading model
make_src has been called!!!
/content/drive/MyDrive/Shared_Attention_for_APE-master/onmt/modules/GlobalAttention.py:150: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  align_vectors = self.sm(align.view(batch*targetL, sourceL))
/usr/local/lib/python3.6/dist-packages/torch/nn/modules/container.py:67: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  input = module(input)
/usr/local/lib/python3.6/dist-packages/torch/tensor.py:309: UserWarning: other is not broadcastable to self, but they have the same number of elements.  Falling back to deprecated pointwise behavior.
  return self.add_(other)
make_src has been called!!!
make_src has been called!!!
make_src has been called!!!
make_src has been called!!!
make_src has been called!!!
make_src has been called!!!
make_src 

#### more epochs and bigger word vector

In [ ]:
! python  ./drive/MyDrive/Shared_Attention_for_APE-master/train.py  -encoder_type  double_encoder  -data  ./drive/MyDrive/Shared_Attention_for_APE-master/output  -save_model  ./drive/MyDrive/test  -word_vec_size  300  -epochs  40 -gpuid 0 -batch_size 32 -optim adadelta

Loading data from './drive/MyDrive/Shared_Attention_for_APE-master/output'
 * vocabulary size. source = 50002; inter = 50002; target = 50004
 * number of training sentences. 77383
 * maximum batch size. 32
Building model...
The double encoder will be needed here
Bidirectional RNN!
DoubleEncNMTModel(
  (encoder): DoubleRNNEncoder(
    (embeddings_src): Embeddings(
      (make_embedding): Sequential(
        (emb_luts): Elementwise(
          (0): Embedding(50002, 300, padding_idx=1)
        )
      )
    )
    (embeddings_inter): Embeddings(
      (make_embedding): Sequential(
        (emb_luts): Elementwise(
          (0): Embedding(50002, 300, padding_idx=1)
        )
      )
    )
    (rnn1): LSTM(300, 250, num_layers=2, dropout=0.3, bidirectional=True)
    (rnn2): LSTM(300, 250, num_layers=2, dropout=0.3, bidirectional=True)
  )
  (decoder): InputFeedRNNDecoder(
    (embeddings): Embeddings(
      (make_embedding): Sequential(
        (emb_luts): Elementwise(
          (0): Embeddin

## APE + baseline

optimizer - adadelta
batch size - 32
word vector - 300


In [ ]:
! python  ./drive/MyDrive/Shared_Attention_for_APE-master/translate.py  -model  ./drive/MyDrive/test_acc_64.55_ppl_14.43_e40.pt  -src  multi_src_modern.txt  -inter multioutputmodern.txt  -tgt multi_pe_modern.txt  -output  ./drive/MyDrive/Shared_Attention_for_APE-master/prediction_file_wordvecBatchAdaModern  -replace_unk

The double encoder will be needed here
Bidirectional RNN!
Loading model
make_src has been called!!!
/content/drive/MyDrive/Shared_Attention_for_APE-master/onmt/modules/GlobalAttention.py:150: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  align_vectors = self.sm(align.view(batch*targetL, sourceL))
/usr/local/lib/python3.6/dist-packages/torch/nn/modules/container.py:67: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  input = module(input)
/usr/local/lib/python3.6/dist-packages/torch/tensor.py:309: UserWarning: other is not broadcastable to self, but they have the same number of elements.  Falling back to deprecated pointwise behavior.
  return self.add_(other)
make_src has been called!!!
make_src has been called!!!
make_src has been called!!!
make_src has been called!!!
make_src has been called!!!
make_src has been called!!!
make_src 

optimizer - adadelta
batch size - 32
epoch - 20


In [ ]:
! python  ./drive/MyDrive/Shared_Attention_for_APE-master/translate.py  -model  ./drive/MyDrive/test_acc_64.58_ppl_13.82_e20.pt  -src  multi_src_modern.txt  -inter  multioutputmodern.txt -tgt multi_pe_modern.txt  -output  ./drive/MyDrive/Shared_Attention_for_APE-master/prediction_adadelta2multi  -replace_unk

The double encoder will be needed here
Bidirectional RNN!
Loading model
make_src has been called!!!
/content/drive/MyDrive/Shared_Attention_for_APE-master/onmt/modules/GlobalAttention.py:150: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  align_vectors = self.sm(align.view(batch*targetL, sourceL))
/usr/local/lib/python3.6/dist-packages/torch/nn/modules/container.py:67: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  input = module(input)
/usr/local/lib/python3.6/dist-packages/torch/tensor.py:309: UserWarning: other is not broadcastable to self, but they have the same number of elements.  Falling back to deprecated pointwise behavior.
  return self.add_(other)
make_src has been called!!!
make_src has been called!!!
make_src has been called!!!
make_src has been called!!!
make_src has been called!!!
make_src has been called!!!
make_src 

## Word vector 300

In [ ]:
! python  ./drive/MyDrive/Shared_Attention_for_APE-master/train.py  -encoder_type  double_encoder  -data  ./drive/MyDrive/Shared_Attention_for_APE-master/output  -save_model  ./drive/MyDrive/test  -word_vec_size  300  -epochs  60 -gpuid 0 -batch_size 32 -optim adadelta

Loading data from './drive/MyDrive/Shared_Attention_for_APE-master/output'
 * vocabulary size. source = 50002; inter = 50002; target = 50004
 * number of training sentences. 77383
 * maximum batch size. 32
Building model...
The double encoder will be needed here
Bidirectional RNN!
DoubleEncNMTModel(
  (encoder): DoubleRNNEncoder(
    (embeddings_src): Embeddings(
      (make_embedding): Sequential(
        (emb_luts): Elementwise(
          (0): Embedding(50002, 300, padding_idx=1)
        )
      )
    )
    (embeddings_inter): Embeddings(
      (make_embedding): Sequential(
        (emb_luts): Elementwise(
          (0): Embedding(50002, 300, padding_idx=1)
        )
      )
    )
    (rnn1): LSTM(300, 250, num_layers=2, dropout=0.3, bidirectional=True)
    (rnn2): LSTM(300, 250, num_layers=2, dropout=0.3, bidirectional=True)
  )
  (decoder): InputFeedRNNDecoder(
    (embeddings): Embeddings(
      (make_embedding): Sequential(
        (emb_luts): Elementwise(
          (0): Embeddin

In [ ]:
! python  ./drive/MyDrive/Shared_Attention_for_APE-master/translate.py  -model  ./drive/MyDrive/test_acc_63.74_ppl_14.87_e13.pt  -src  ./drive/MyDrive/test/sp_test.src  -inter  ./drive/MyDrive/test/pl_test.mt -tgt ./drive/MyDrive/test/pl_test.pe  -output  ./drive/MyDrive/Shared_Attention_for_APE-master/prediction_file_worvec2  -replace_unk

The double encoder will be needed here
Bidirectional RNN!
Loading model
make_src has been called!!!
/content/drive/MyDrive/Shared_Attention_for_APE-master/onmt/modules/GlobalAttention.py:150: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  align_vectors = self.sm(align.view(batch*targetL, sourceL))
/usr/local/lib/python3.6/dist-packages/torch/nn/modules/container.py:67: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  input = module(input)
/usr/local/lib/python3.6/dist-packages/torch/tensor.py:309: UserWarning: other is not broadcastable to self, but they have the same number of elements.  Falling back to deprecated pointwise behavior.
  return self.add_(other)
make_src has been called!!!
make_src has been called!!!
make_src has been called!!!
make_src has been called!!!
make_src has been called!!!
make_src has been called!!!
make_src 